In [67]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
from arcpy import env
from arcpy.sa import *
import math
from arcpy import metadata as md
import inspect
import datetime
import shutil
import csv

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

Created by Zac Canter for CBI 3/21

Required input data: Expanded WUI (to be created in another notebook), Roads and Gaps (the areas between parcels and SBC county shapefile)

**Overall process:**
- 1) Select the parcels that intersect the expanded WUI
- 2) Subdivide those that are larger than 150 acres into 100 acre subdivisions
- 3) Select parcels that are less than 10 acres, and dissolve boundaries
- 4) Of those smaller dissolved boundaries, take those that are over 150 acres and subdivide them into 10 acre subdivisions
- 5) Subdivide the roads and gap layers into 10 acre parcel subdivisions
- 6) Merge the road and gap subdivisons, the 100 acre subdivisions, the 10 acre subdivisions, and all the remaining geometry
- 7) Elimate geometry that is less than 1 acre
- 8) Add APN and GRID_ID attribute

In [68]:
in_data = ["Expanded_WUI.shp", "Roads_Gaps.shp", "SBC_Boundary.shp", "SBC_Parcels.shp"]

Create the geodatabase

In [69]:
gdb_name = "RPP_ReportingUnits.gdb"
arcpy.ResetEnvironments()
arcpy.env.overwriteOutput = True
arcpy.CreateFileGDB_management("./", gdb_name)
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

Copy all the starting data into the geodatabase and reproject it

In [70]:
for v in in_data:
    path = os.path.join(THIS_FOLDER, "in_data/{}".format(v))
    name = v.split(".")[0]
    arcpy.CopyFeatures_management(path, name)
    crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
    arcpy.Project_management(name, "{}_rpj".format(name), crs)
    arcpy.Delete_management(name)

Get the parcels that intersect with the WUI

In [71]:
selection = arcpy.SelectLayerByLocation_management("SBC_Parcels_rpj", "INTERSECT", "Expanded_WUI_rpj")
WUI_Intersecting_Parcels = arcpy.CopyFeatures_management(selection, "WUI_Intersecting_Parcels")

Create the variables for the cutoffs

In [72]:
LgCutoff = 150
SmCutoff = 10
XSmCutoff = 1
BigBlock = 100
SmBlock = 10
acre = 43560

Select the parcels that are less than or equal to 150 acres and greater than or equal to 10 acres and add those to the reporting units list

In [73]:
LTE_Lg_GTE_Sm = arcpy.management.SelectLayerByAttribute(WUI_Intersecting_Parcels, "NEW_SELECTION", "Shape_Area <= {} And Shape_Area >= {}".format(LgCutoff * acre, SmCutoff * acre), None)
arcpy.CopyFeatures_management(LTE_Lg_GTE_Sm, "LTE_Lg_GTE_Sm")
reporting_units = ["LTE_Lg_GTE_Sm"]

Select the parcels that are larger than 150 acres and subdivide them into 100 acre subdivisions, add those to the reporting units list

In [74]:
GT_Lg = arcpy.management.SelectLayerByAttribute(WUI_Intersecting_Parcels, "NEW_SELECTION", "Shape_Area > {}".format(LgCutoff * acre), None)
GT150_SubDiv100 = arcpy.management.SubdividePolygon(GT_Lg, "GT150_SubDiv100", "EQUAL_AREAS", None, "{} SquareFeet".format(BigBlock * acre), None, 0, "STACKED_BLOCKS")
reporting_units.append("GT150_SubDiv100")

Now we process parcels that are less than 10 acres. We will first select them, and dissolve their boundaries. The resulting ones that are larger than 150 acres are subdivided into 10 acre subdivisions, then these are all added to the reporting units list

In [75]:
LT_Sm = arcpy.management.SelectLayerByAttribute(WUI_Intersecting_Parcels, "NEW_SELECTION", "Shape_Area < {}".format(SmCutoff * acre), None)
LT10_Dis = arcpy.gapro.DissolveBoundaries(LT_Sm, "LT10_Dis", "SINGLE_PART", None, None, None)

GT_Lg_Dis = arcpy.management.SelectLayerByAttribute(LT10_Dis, "NEW_SELECTION", "Shape_Area > {}".format(LgCutoff * acre), None)
GT_Lg_Dis_SubDiv = arcpy.management.SubdividePolygon(GT_Lg_Dis, "GT_Lg_Dis_SubDiv", "EQUAL_AREAS", None, "{} SquareFeet".format(SmBlock * acre), None, 0, "STACKED_BLOCKS")
reporting_units.append("GT_Lg_Dis_SubDiv")

In [76]:
LTE_Lg = arcpy.management.SelectLayerByAttribute(LT10_Dis, "NEW_SELECTION", "Shape_Area <= {}".format(LgCutoff * acre), None)
arcpy.CopyFeatures_management(LTE_Lg, "LTE_Lg")
reporting_units.append("LTE_Lg")

Subdivide roads and gaps into 10 acre subdivisions and add those to the reporting units list

In [77]:
arcpy.management.SubdividePolygon("Roads_Gaps_rpj", "Roads_Gaps_rpj_SubDiv", "EQUAL_AREAS", None, "{} SquareFeet".format(SmBlock * acre), None, 0, "STACKED_BLOCKS")
reporting_units.append("Roads_Gaps_rpj_SubDiv")

Merge the reporting units list and convert them to single part for elimination process

In [78]:
arcpy.Merge_management(reporting_units, "Merge1")
arcpy.management.MultipartToSinglepart("Merge1", "Merge1_SP")

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\RPP_ReportingUnits.gdb\\Merge1_SP'>

Eliminate geometry that is smaller than 1 acre

In [79]:
selection = arcpy.management.SelectLayerByAttribute("Merge1_SP", "NEW_SELECTION", "Shape_Area < {}".format(XSmCutoff * acre), None)
arcpy.management.Eliminate(selection, "Merge1_SP_Elim", "LENGTH", "", None)

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\RPP_ReportingUnits.gdb\\Merge1_SP_Elim'>

The merged layer is beyond the extent of the WUI parcels so we need to aggregate those parcels to get an outline shape then we can use them to clip the roads and gaps appropriately. First we aggregate the polygons within 500 feet of each other, we also set the minimum hole size to 100 acres.

In [88]:
arcpy.cartography.AggregatePolygons("WUI_Intersecting_Parcels", "WUI_Parcel_Boundary", "500 Feet", "0 SquareFeet", "4356000 SquareFeet", "NON_ORTHOGONAL", None, "WUI_Parcel_Boundary_Tbl", None)
arcpy.analysis.Clip("Merge1_SP_Elim", "WUI_Parcel_Boundary", "Reporting_Units", None)

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\RPP_ReportingUnits.gdb\\Reporting_Units'>

Remove all fields besides APN and mandatory ones

In [89]:
fieldObs = arcpy.ListFields("Reporting_Units")  

fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  

print(fieldNames)

['OBJECTID', 'Shape', 'apn', 'Shape_Length', 'Shape_Area']


In [86]:
FCfields = [f.name for f in arcpy.ListFields("Merge1_SP_Elim")]
DontDeleteFields = ['Shape_Length', 'Shape_Area', 'OBJECTID', 'apn', 'Shape']
fields2Delete = list(set(FCfields) - set(DontDeleteFields))
arcpy.DeleteField_management("Merge1_SP_Elim", fields2Delete)

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\RPP_ReportingUnits.gdb\\Merge1_SP_Elim'>